## Text Analytics Group Assignment 2
### ArjunVarma_AnyingLi_YuanzhuoPeng_JiaqiuWang_NimishAmlathe

In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
df['rating'] = df['stars'].map(lambda x: 'High' if x >3  else 'Low')
df[:5]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This location is out of business. I drove by i...,Low
1,2,2,2,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,= = = = = = CLOSED = = = = = =This JB s locati...,Low
2,4,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This is just a basic (albeit mini) chain greas...,High
3,3,0,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Whenever I offer to take my mom out to lunch s...,Low
4,3,7,9,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,If I say it wasn t as bad as I was expecting i...,Low


In [3]:
df.columns.values

array(['stars', 'votes_cool', 'votes_funny', 'votes_useful', 'Cheap',
       'Moderate', 'Expensive', 'VeryExpensive', 'American', 'Chinese',
       'French', 'Japanese', 'Indian', 'Italian', 'Greek', 'Mediterranean',
       'Mexican', 'Thai', 'Vietnamese', 'Others', 'Review', 'rating'], dtype=object)

#### Task A. Ignore the text (reviews) and run a classification model with the numeric data (you can use standard methods like logistic regression, k-nearest neighbors or anything else). What is the best accuracy of your model with numeric data?

In [4]:
#create design matrices
Y, X = dmatrices('rating ~ votes_cool+votes_funny+votes_useful+Cheap+Moderate+Expensive+VeryExpensive+\
                 American+Chinese+French+Japanese+Indian+Italian+Greek+Mediterranean+\
                 Mexican+Thai+Vietnamese+Others',df,return_type="dataframe")
y = Y['rating[High]']

#Split the data into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [5]:
#knn
from sklearn import neighbors
modelknn = neighbors.KNeighborsClassifier(n_neighbors=15, weights='uniform', p=2)
modelknn.fit(X_train,y_train)

from sklearn import metrics
prediction_on_training = modelknn.predict(X_train)
print "Train set accuracy score:", metrics.accuracy_score(y_train, prediction_on_training)

predicted_classes = modelknn.predict(X_test)
print "Test set accuracy score:", metrics.accuracy_score(y_test, predicted_classes)

Train set accuracy score: 0.687406243303
Test set accuracy score: 0.6665


In [6]:
#logistic regression
from sklearn.linear_model import LogisticRegression
modellog = LogisticRegression()
modellog.fit(X_train, y_train)

from sklearn import metrics
prediction_on_training = modellog.predict(X_train)
print "Train set accuracy score:", metrics.accuracy_score(y_train, prediction_on_training)

predicted_classes = modellog.predict(X_test)
print "Test set accuracy score:", metrics.accuracy_score(y_test, predicted_classes)

Train set accuracy score: 0.68512036574
Test set accuracy score: 0.685


In [7]:
#naive bayes
from sklearn import naive_bayes
modelnby = naive_bayes.MultinomialNB()
modelnby.fit(X_train, y_train)

from sklearn import metrics
prediction_on_training = modelnby.predict(X_train)
print "Train set accuracy score:", metrics.accuracy_score(y_train, prediction_on_training)

predicted_classes = modelnby.predict(X_test)
print "Test set accuracy score:", metrics.accuracy_score(y_test, predicted_classes)

Train set accuracy score: 0.677619829988
Test set accuracy score: 0.6795


So the best accuracy with using only numeric data for prediction is about 68.5% on test data with logistic regression model.

#### Task B. Perform a supervised classification on a subset of the corpus using the reviews only. You can write your code in Python or R. What accuracy do you get from this text mining exercise?

In [8]:
import nltk
import operator
import re

In [9]:
#bag-of-words approach
review  = df['Review'][:1000]
corpus = review.str.cat(sep=' ')

corpus = corpus.decode('utf-8')

corpus_words = nltk.word_tokenize(corpus.lower())
corpus_words = [word for word in corpus_words if word.isalpha()==True]

In [12]:
# Create a function that counts the frequency of each token in a document
def document_features(document):
    document_words = set(document)
    features = {}
    for word in corpus_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

df['token'] = ""
train = df[['Review','token','rating']][:1000]

# Calculate the document_features in each document
from __future__ import unicode_literals
for i in range(len(train)):
    words = re.sub(r'[^\w\s]','',train.Review[i])
    token = nltk.word_tokenize(words.lower().decode('utf-8'))
    token = [w for w in token if w.isalpha()==True]
    feature = document_features(token)
    train.token[i] = feature

In [14]:
# Create the feature set and run the model
train['feature_sets'] = zip(train.token, train.rating)
featuresets = train.feature_sets.tolist()

#set train and test set
from sklearn.cross_validation import train_test_split
train_set, test_set = train_test_split(featuresets, test_size=0.3, random_state=1)

classifier = nltk.NaiveBayesClassifier.train(train_set)

#get accuracy score
print 'The accuracy for train set is ', nltk.classify.accuracy(classifier, train_set)
print 'The accuracy for test set is ', nltk.classify.accuracy(classifier, test_set)

The accuracy for train set is  0.898571428571
The accuracy for test set is  0.696666666667


In [15]:
# Confusion matrix
test_mod = [x[0] for x in test_set]
predict = classifier.classify_many(test_mod)
actual = [x[1] for x in test_set]
cm = nltk.ConfusionMatrix(actual, predict)
print (cm.pretty_format(sort_by_count=True, show_percents=True))

     |      H        |
     |      i      L |
     |      g      o |
     |      h      w |
-----+---------------+
High | <58.7%> 14.3% |
 Low |  16.0% <11.0%>|
-----+---------------+
(row = reference; col = test)



So the accuracy with using only review data for prediction is about 69.7% on test data.